In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import selenium.webdriver
from datetime import date
import selenium
import pandas as pd
from selenium import webdriver

from time import sleep
import time
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementNotInteractableException
import warnings
warnings.filterwarnings('ignore')

In [2]:
#extracting path of the webdriver
driver=webdriver.Chrome(r'C:\Users\yashn\Downloads\chromedriver_win32\chromedriver.exe') 

In [3]:
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(4)
#clicking on cancel buttopn 
driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']").click()
srch_items = ['laptops', 'Phones','smart watches']
title = []
review_text = []
ratings = []

In [4]:
def scrap():    
        for i in driver.find_elements_by_xpath("//div[@class='t-ZTKy']"):
            review_text.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']"):
            title.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):
            ratings.append(i.text)
        return
urls=[]
for i in srch_items:
    # Find the search bar
    srchBar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    srchBar.clear()
    srchBar.send_keys(i)
    # Clicking on the search button
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    page = []
    for i in driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:4]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))
len(urls)            

288

In [5]:
for i in urls:
    driver.get(i)
    time.sleep(3)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,4000)")
        time.sleep(1)
    # Clicking on all reviews
    try:
        btn=driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(2)
    except NoSuchElementException:
        pass

    scrap()        
    try:
        n_page=driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(2)
            scrap()
    except: continue
      

In [6]:
len(ratings), len(review_text), len(title)

(25911, 25911, 25911)

In [7]:
# Creating a dataframe
data = list(zip(title,review_text,ratings))
df = pd.DataFrame(data, columns = ["Review_title","Review_text","Ratings"])
df

,Review_title,Review_text,Ratings
0,Perfect product!,One of the best Laptop in the market for home ...,5
1,Just wow!,Excellent & far far better than HP laptops. Ve...,5
2,Simply awesome,Best budget laptop if u are student then this ...,5
3,Mind-blowing purchase,Lapt lennovo is very good excellent product he...,5
4,Very Good,"I had purchased this product 3 weeks ago, firs...",4
...,...,...,...
25906,Pretty good,Writing this review after 45 days of use.\n.\n...,4
25907,Awesome,The best product ever in that range prize very...,5
25908,Wonderful,Great watch in budget track step quite good in...,5
25909,Terrific,The best smartwatch available right now with a...,5


In [11]:
# writing to Excel
datatoexcel = pd.ExcelWriter('Ratings_review_data.xlsx')
  
# write DataFrame to excel
df.to_excel(datatoexcel)
  
# save to excel
datatoexcel.save()